In [ ]:
import numpy as np
import pandas as pd
import re # Used for the conversion of "r##_c##" in only the numbers --> TODO: check where it comes from
from helpers import *
from play_with_data import *
from pre_processing import *
from matrix_factorization import *
from cross_validation import *


# Done before the exercise

In [ ]:
#********* Load the given data **********
r_c, x = load_data_old('./data_train.csv')

In [ ]:
#********* Creation of a matrix of the data ********
nUser = 10000
nItem = 1000
data = np.zeros([nUser, nItem]) # These numbers were given

for ind, i in enumerate(r_c): # Loop over all the ID 
    data[int(re.findall('\d+', i)[0])-1, int(re.findall('\d+', i)[1])-1] = x[ind] # Use the information in the ID (row, col) to create the matrix




In [ ]:
df = pd.DataFrame(data)
a = []
for i in range(df.shape[0]):
    a.append(len(np.where(df.loc[i] != 0)[0]))
#print(a)
#print(len(a))
print(np.std(a))
print(np.mean(a))
#display(df)

In [ ]:
#********** Data preview ************
# Check if there is any missed data 
# It was told us that we have the data from 10'000 users for 1000 films, but we don't have all these data
info_general(nUser, nItem, x, data)
print('\n')
info_ratings(data)


# Done after the exercise

In [ ]:
#******** Creation of a sparse matrix of the data **********
ratings = load_data('./data_train.csv') 
print(ratings.shape) # rows are "users" and "items" are the columns

In [ ]:
#***** Data preview *********
num_items_per_user, num_users_per_item = plot_raw_data(ratings) # Original code is from the course, ex10 'plots.py'
print("Maximum number of items per user:\t{}\nMinimum number of items per user:\t{}\n".format(np.max(num_items_per_user), np.min(num_items_per_user)))
print("Maximum number of users per item:\t{}\nMinimum number of users per item:\t{}".format(np.max(num_users_per_item), np.min(num_users_per_item)))

In [ ]:
#********** Pre-processing **********
min_num_ratings = 10 # or 15 this is based on the information given above
ratings_valid = valid_ratings(ratings, num_items_per_user, num_users_per_item,min_num_ratings)
train, test = split_data(ratings_valid) # This will put 90% of the items for the users that have at least one non-zero entry and the 10% in test


In [ ]:
def build_k_indicesI(train_set, k_fold,seed):
    
    num_row = train_set.shape[0]
    interval = int(num_row/k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval]
                 for k in range(k_fold)]
    
    return np.array(k_indices)

a = build_k_indices(train, 10,1)

In [ ]:
def cross_validationI(train_set, k_indices, k):
    """return the loss of ridge regression."""
    index_set = np.arange(train_set.shape[0])
    tr_index = set(k_indices[k,:]).symmetric_difference(index_set)
    tr_index = np.array(list(tr_index)) # Convert set to array
    
    cv_te = train_set[k_indices[k,:],:]
    cv_tr = train_set[tr_index,:]
    
    return cv_tr, cv_te

In [ ]:
def cross_validation_applicationI(train_set,default,k_fold,gamma, num_features, lambda_user, lambda_movie,seed):

    # split data in k fold
    k_indices = build_k_indices(train_set, k_fold, seed)
    validation_rmse = []
    validation_error = []
    for i in range(k_fold):
        
        cv_tr, cv_te = cross_validation(train_set, k_indices, i)
        rmse = matrix_factorization_SGD(cv_tr, cv_te, default ,gamma, num_features, lambda_user, lambda_movie)
        validation_rmse.append(rmse)
    
    validation_error = validation_rmse.mean()
    return validation_error
        

In [ ]:
cross_validation_application(train ,False ,4,0.01, 20, 0.2, 0.3,1)

In [ ]:
#********* Try ************
#rmse = rmse_movie_mean(train, test)
rmse = matrix_factorization_SGD(train, test)

In [ ]:
rmse

# Done so far
- Preview of the data
- Pre-processing:
    - Choosing only the "valid ratings", the users and items that contains more than min_num_ratings
    - Splitting the data in test and train, by choosing 90% of the ratings from the valid_ratings and only the non-zeros values
    